In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import os
import sys
sys.path.append('/Users/tecork/Downloads/CarDpy-master')
from cardpy.Sample_Data.Data_Paths                  import *
from cardpy.Data_Import                             import *
from cardpy.Data_Sorting                            import *
from cardpy.Data_Saving                             import *
from cardpy.Data_Processing.Gibbs                   import *
from cardpy.Data_Processing.Registration            import *
from cardpy.Data_Processing.Rejection               import *
from cardpy.Data_Processing.Respiratory             import *
from cardpy.Data_Processing.Diffusivity             import *
from cardpy.Data_Processing.Averaging               import *
from cardpy.Data_Processing.Denoising               import *
from cardpy.Data_Processing.Interpolation           import *
from cardpy.Data_Processing.Segmentation_Matrix_DTI import *

from cardpy.GUI_Tools                     import *
from cardpy.Colormaps                     import *
# import cardpy

In [ ]:
data_type                 = 'NifTis'         # Input data type for sample Data (DICOM or NifTis)

### User Settings
DICOM_reader_info         = 'ON'             # Flag to read from DICOM folder
operation_type            = 'Magnitude'      #

Gibbs_Mode                = 'ON'             # Flag to register data

Rejection_Mode            = 'ON'             # Flag to reject and replace bad acquisitions
NRMSE_Threshold           = 0.75
NSSIM_Threshold           = 0.75 
rejection_zoom            = 'ON'
rejection_IntEACT_zoom    = 'ON'
rejection_organ           = 'Heart'

Respiratory_Mode          = 'ON'             # Flag to reject and replace bad acquisitions
respiratory_zoom          = 'ON'
respiratory_IntERACT_zoom = 'ON'
respiratory_organ         = 'Stomach'

Registration_Mode         = 'ON'             # Flag to register data
registration_algorithm    = 'Affine'         # Type of registration to perform
temporary_denoising       = 'OFF'

ADC_Filter_Mode          = 'ON'
Averaging_Mode           = 'ON'             # Flag to average data

Denoising_Mode           = 'ON'             # Flag to average data
number_of_coils          = 20
denoising_algorithm      = 'LocalPCA'       # Type of denoising to perform

Interpolation_Mode       = 'ON'             # Flag to interpolate data

Extended_Matrix_Mode     = 'ON'             # Flag to interpolate data

E1_Mode                  = 'ON'


if registration_algorithm == 'Affine':
    sub_registration = 'Rigid'
if registration_algorithm == 'Rigid':
    sub_registration = 'Translation'
if registration_algorithm == 'Translation':
    sub_registration = 'Translation'

In [ ]:
### Sample Data Demo
if data_type == 'DICOM':
    DICOM_Folder = sample_DICOMs()
    print(DICOM_Folder)
    [matrix_stacked, b_vals_stacked, b_vecs_stacked, Header] = DICOM_Reader(DICOM_Folder, info='ON')
if data_type == 'NifTis':
    [NifTi_path, b_values_path, b_vectors_path, header_path] = sample_NifTis()
    print(NifTi_path)
    print(b_values_path)
    print(b_vectors_path)
    print(header_path)
    [matrix_stacked, b_vals_stacked, b_vecs_stacked, Header, _, _] = NifTi_Reader(NifTi_path, b_values_path, b_vectors_path, header_path)

temporary_matrix_stacked = matrix_stacked
temporary_bvals_stacked  = b_vals_stacked
temporary_bvecs_stacked  = b_vecs_stacked
out_path                 = sample_Output_Folder()

### In order to use this script with your own data, 
### please define the variables in the print statments of this cell.
### - DICOM_Folder for DICOMs
### - NifTi_path, b_values_path, b_vectors_path, and header_path for NifTis


In [ ]:
########## DICOM Import Module #################################################################################################################
counter        = 0
counter_string = str(counter)
folder         = counter_string.zfill(2) + '_Original'
[temporary_matrix_sorted,  temporary_bvals_sorted,  temporary_bvecs_sorted]          = stacked2sorted(temporary_matrix_stacked, 
                                                                                                      temporary_bvals_stacked, 
                                                                                                      temporary_bvecs_stacked)
output_path = os.path.join(out_path, folder)
file_name   = 'Original'
Save_Diffusion_Image_Data(output_path, file_name, Header, temporary_matrix_sorted, temporary_bvals_sorted, temporary_bvecs_sorted)
del counter_string, folder, output_path
del temporary_matrix_stacked, temporary_bvals_stacked, temporary_bvecs_stacked
################################################################################################################################################
########## Gibb's Ringing Module ###############################################################################################################
if Gibbs_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Unrung'
    print('Gibbs ringing removal mode is on.')
    ### Step 1: Perform Gibb's ringing removal on temporary sorted matrix
    [unrung_matrix_sorted, unrung_bvals_sorted, unrung_bvecs_sorted] = unrung(temporary_matrix_sorted, 
                                                                              temporary_bvals_sorted, 
                                                                              temporary_bvecs_sorted, 
                                                                              operation_type = operation_type)
    ### Step 2: Save unrung matrix
    output_path = os.path.join(out_path, folder)
    Save_Diffusion_Image_Data(output_path, 'Unrung', Header, unrung_matrix_sorted, unrung_bvals_sorted, unrung_bvecs_sorted)
    ### Step 3: Assign unrung variables to temporary variables to be overwritten
    temporary_matrix_sorted = unrung_matrix_sorted
    temporary_bvals_sorted  = unrung_bvals_sorted
    temporary_bvecs_sorted  = unrung_bvecs_sorted
    ### Step 4: Delete Variables
    del counter_string, folder, output_path
    del unrung_matrix_sorted, unrung_bvals_sorted, unrung_bvecs_sorted
if Gibbs_Mode == 'OFF':
    print('Gibbs ringing removal mode is off.')
################################################################################################################################################
########## Rejection Module ####################################################################################################################
if Rejection_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Rejected'
    print('Rejection mode is on.')
    ### Step 1: Perform registration on temporary sorted matrix
    print('Re-registering data prior to data rejection.')
    [registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted] = register(temporary_matrix_sorted, 
                                                                                            temporary_bvals_sorted, 
                                                                                            temporary_bvecs_sorted,
                                                                                            registration_algorithm = sub_registration,
                                                                                            temporary_denoising    = temporary_denoising,
                                                                                            operation_type         = operation_type)
    ### Step 2: Perform rejection on registered sorted matrix
    [rejected_matrix_sorted, rejected_bvals_sorted, rejected_bvecs_sorted, Slice_Coordinates] = shot_rejection(registered_matrix_sorted, 
                                                                                                               registered_bvals_sorted, 
                                                                                                               registered_bvecs_sorted,
                                                                                                               NRMSE_threshold = NRMSE_Threshold, 
                                                                                                               NSSIM_threshold = NSSIM_Threshold, 
                                                                                                               zoom            = rejection_zoom, 
                                                                                                               IntERACT_zoom   = rejection_IntEACT_zoom, 
                                                                                                               operation_type  = operation_type)

    ### Step 3: Save registered stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Save_Diffusion_Image_Data(output_path, 'Rejected', Header, rejected_matrix_sorted, rejected_bvals_sorted, rejected_bvecs_sorted)
    ### Step 4: Assign registered stacked variables to temporary variables to be overwritten
    temporary_matrix_sorted = rejected_matrix_sorted
    temporary_bvals_sorted  = rejected_bvals_sorted
    temporary_bvecs_sorted  = rejected_bvecs_sorted
    ### Step 5: Delete Variables
    del counter_string, folder, output_path
    del rejected_matrix_sorted, rejected_bvals_sorted, rejected_bvecs_sorted
    del registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted
if Rejection_Mode == 'OFF':
    print('Rejection mode is off.')
################################################################################################################################################
########## Respiratory Module ##################################################################################################################
if Respiratory_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Respiratory_Sorted'
    print('Respiratory sorting mode is on.')
    ### Step 1: Perform registration on temporary sorted matrix
    print('Re-registering data prior data reordering.')
    [registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted] = register(temporary_matrix_sorted,
                                                                                            temporary_bvals_sorted, 
                                                                                            temporary_bvecs_sorted, 
                                                                                            registration_algorithm = registration_algorithm,
                                                                                            temporary_denoising    = temporary_denoising,
                                                                                            operation_type         = operation_type)
    ### Step 2: Perform respiratory sorting on rejected sorted matrix
    [respiratory_matrix_sorted, respiratory_bvals_sorted, respiratory_bvecs_sorted, SLICE_Coordinates] = respiratory_sorting(registered_matrix_sorted,
                                                                                                                             registered_bvals_sorted, 
                                                                                                                             registered_bvecs_sorted,
                                                                                                                             zoom           = respiratory_zoom,
                                                                                                                             IntERACT_zoom  = respiratory_IntERACT_zoom, 
                                                                                                                             organ          = respiratory_organ,
                                                                                                                             operation_type = operation_type)

    ### Step 3: Save registered stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Save_Diffusion_Image_Data(output_path, 'Respiratory_Ordered', Header, respiratory_matrix_sorted, respiratory_bvals_sorted, respiratory_bvecs_sorted)
    ### Step 4: Assign registered stacked variables to temporary variables to be overwritten
    temporary_matrix_sorted = respiratory_matrix_sorted
    temporary_bvals_sorted  = respiratory_bvals_sorted
    temporary_bvecs_sorted  = respiratory_bvecs_sorted
    ### Step 5: Delete Variables
    del counter_string, folder, output_path
    del respiratory_matrix_sorted, respiratory_bvals_sorted, respiratory_bvecs_sorted
    del registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted
if Rejection_Mode == 'OFF':
    print('Rejection mode is off.')
################################################################################################################################################
########## Registration Module #################################################################################################################
if Registration_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Registered'
    print('Registration mode is on.') 
    ### Step 1: Perform registration on temporary sorted matrix
    [registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted] = register(temporary_matrix_sorted,
                                                                                            temporary_bvals_sorted, 
                                                                                            temporary_bvecs_sorted, 
                                                                                            registration_algorithm = registration_algorithm,
                                                                                            temporary_denoising    = temporary_denoising,
                                                                                            operation_type         = operation_type)
    ### Step 2: Save registered stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Save_Diffusion_Image_Data(output_path, 'Registered', Header, registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted)
    ### Step 3: Assign registered stacked variables to temporary variables to be overwritten
    temporary_matrix_sorted = registered_matrix_sorted
    temporary_bvals_sorted  = registered_bvals_sorted
    temporary_bvecs_sorted  = registered_bvecs_sorted
    ### Step 4: Delete Variables
    del counter_string, folder, output_path
    del registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted
if Registration_Mode == 'OFF':
    print('Registration mode is off.')
################################################################################################################################################
########## ADC Filter Module ###################################################################################################################
if ADC_Filter_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Diffusivity_Filtered'
    print('ADC Filter mode is on.')
    ### Step 1: Perform averaging on temporary sorted matrix
    [filtered_matrix_sorted, filtered_bvals_sorted, filtered_bvecs_sorted] = ADC_Filter(temporary_matrix_sorted, 
                                                                                        temporary_bvals_sorted, 
                                                                                        temporary_bvecs_sorted,
                                                                                        operation_type = operation_type)
    ### Step 2: Save averaged stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Save_Diffusion_Image_Data(output_path, 'ADC_Filtered', Header, filtered_matrix_sorted, filtered_bvals_sorted, filtered_bvecs_sorted)
    ### Step 3: Assign registered stacked variables to temporary variables to be overwritten
    temporary_matrix_sorted = filtered_matrix_sorted
    temporary_bvals_sorted  = filtered_bvals_sorted
    temporary_bvecs_sorted  = filtered_bvecs_sorted
    ### Step 4: Delete Variables
    del counter_string, folder, output_path
    del filtered_matrix_sorted, filtered_bvals_sorted, filtered_bvecs_sorted 
if Averaging_Mode == 'OFF':
    print('Averaging mode is off.')


########## Averaging Module ####################################################################################################################
if Averaging_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Averaged'
    print('Averaging mode is on.')
    ### Step 1: Perform averaging on temporary sorted matrix
    [averaged_matrix_sorted, averaged_bvals_sorted, averaged_bvecs_sorted] = average(temporary_matrix_sorted, 
                                                                                     temporary_bvals_sorted, 
                                                                                     temporary_bvecs_sorted,
                                                                                     operation_type = operation_type)
    print('Re-registering data after data averaging.')
    ### Step 2: Perform registration on average sorted matrix
    [registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted] = register(averaged_matrix_sorted,
                                                                                            averaged_bvals_sorted, 
                                                                                            averaged_bvecs_sorted, 
                                                                                            registration_algorithm = registration_algorithm,
                                                                                            temporary_denoising    = temporary_denoising,
                                                                                            operation_type         = operation_type)
    ### Step 3: Save averaged stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Save_Diffusion_Image_Data(output_path, 'Averaged', Header, registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted)
    ### Step 4: Assign registered stacked variables to temporary variables to be overwritten
    temporary_matrix_sorted = registered_matrix_sorted
    temporary_bvals_sorted  = registered_bvals_sorted
    temporary_bvecs_sorted  = registered_bvecs_sorted
    ### Step 5: Delete Variables
    del counter_string, folder, output_path
    del averaged_matrix_sorted, averaged_bvals_sorted, averaged_bvecs_sorted 
    del registered_matrix_sorted, registered_bvals_sorted, registered_bvecs_sorted
if Averaging_Mode == 'OFF':
    print('Averaging mode is off.')
################################################################################################################################################
########## Denoising Module ####################################################################################################################
if Denoising_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Denoised'
    print('Denoising mode is on.')
    ### Step 1: Perform denoising on temporary stacked matrix
    [denoised_matrix_sorted, denoised_bvals_sorted, denoised_bvecs_sorted] = denoise(temporary_matrix_sorted, 
                                                                                     temporary_bvals_sorted, 
                                                                                     temporary_bvecs_sorted,
                                                                                     denoising_algorithm = denoising_algorithm,
                                                                                     operation_type      = operation_type)
    ### Step 2: Save denoised stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Save_Diffusion_Image_Data(output_path, 'Denoised', Header, denoised_matrix_sorted, denoised_bvals_sorted, denoised_bvecs_sorted)
    ### Step 3: Assign registered stacked variables to temporary variables to be overwritten
    temporary_matrix_sorted = denoised_matrix_sorted
    temporary_bvals_sorted  = denoised_bvals_sorted
    temporary_bvecs_sorted  = denoised_bvecs_sorted
    ### Step 4: Delete Variables
    del counter_string, folder, output_path
    del denoised_matrix_sorted, denoised_bvals_sorted, denoised_bvecs_sorted
if Denoising_Mode == 'OFF':
    print('Denoising mode is off.')
################################################################################################################################################
########## Interpolation Module ################################################################################################################
if Interpolation_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Interpolated'
    print('Interpolating mode is on.')
    ### Step 1: Perform averaging on temporary sorted matrix
    [interpolated_matrix_sorted, interpolated_bvals_sorted, interpolated_bvecs_sorted] = zero_filled(temporary_matrix_sorted, 
                                                                                                     temporary_bvals_sorted, 
                                                                                                     temporary_bvecs_sorted,
                                                                                                     operation_type = operation_type)
    ### Step 2: Save averaged stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Header['X Resolution'] = Header['X Resolution'] / 2
    Header['Y Resolution'] = Header['Y Resolution'] / 2    
    Save_Diffusion_Image_Data(output_path, 'Interpolated', Header, interpolated_matrix_sorted, interpolated_bvals_sorted, interpolated_bvecs_sorted)
    ### Step 3: Assign registered stacked variables to temporary variables to be overwritten
    temporary_matrix_sorted = interpolated_matrix_sorted
    temporary_bvals_sorted  = interpolated_bvals_sorted
    temporary_bvecs_sorted  = interpolated_bvecs_sorted
    ### Step 4: Delete Variables
    del counter_string, folder, output_path
    del interpolated_matrix_sorted, interpolated_bvals_sorted, interpolated_bvecs_sorted
if Interpolation_Mode == 'OFF':
    print('Interpolating mode is off.')
################################################################################################################################################
########## Extended Matrix Module ##############################################################################################################
if Extended_Matrix_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Extended_Matrix'
    print('Extended matrix mode is on.')
    ### Step 1: Perform 
    [segmentation_matrix, segmentation_bvals, segmentation_bvecs] = DTI_segmentation_matrix(temporary_matrix_sorted, 
                                                                                            temporary_bvals_sorted, 
                                                                                            temporary_bvecs_sorted)
    [segmentation_matrix, segmentation_bvals, segmentation_bvecs] = stacked2sorted(segmentation_matrix, 
                                                                                   segmentation_bvals, 
                                                                                   segmentation_bvecs)
    ### Step 2: Save averaged stacked matrix (DiPy format)
    output_path = os.path.join(out_path, folder)
    Save_Diffusion_Image_Data(output_path, 'Extended_Matrix', Header, segmentation_matrix, segmentation_bvals, segmentation_bvecs)
    ### Step 3: Delete Variables
    del counter_string, folder, output_path
    del segmentation_matrix, segmentation_bvals, segmentation_bvecs
if Extended_Matrix_Mode == 'OFF':
    print('Extended matrix mode is off.')
################################################################################################################################################
########## Primary Eigenvector Map Module ######################################################################################################
if E1_Mode == 'ON':
    counter        = counter + 1
    counter_string = str(counter)
    folder         = counter_string.zfill(2) + '_Primary_Eigenvector'
    output_path    = os.path.join(out_path, folder)
    print('Primary Eigenvector mode is on.')
    Save_Primary_Eigenvector_Data(output_path, Header, temporary_matrix_sorted, temporary_bvals_sorted, temporary_bvecs_sorted)
if E1_Mode == 'OFF':
    print('Primary eigenvector mode is off.')
################################################################################################################################################